In [ ]:
import sys
sys.path.append("./code/")

from model import RNNBinPacking
class RNNBinPackingWithSigmoid(RNNBinPacking):
    def forward(self, x):
        x = super().forward(x)
        return x.sigmoid()

import json
import torch
from datetime import datetime

In [ ]:
with open('args_info.json', 'r', encoding='utf-8') as f:
    args_dict = json.load(f)

hidden_size = args_dict['dim']
nhead = args_dict['head']
num_transformer_layers = args_dict['transformer_layers']
num_rnn_layers = args_dict['rnn_layers']
num_fc_neurons = hidden_size
d_ff = 4 * hidden_size
device = torch.device("cuda:0" if args_dict['gpu'] and torch.cuda.is_available() else "cpu")

model = RNNBinPackingWithSigmoid(hidden_size, nhead, num_transformer_layers, num_rnn_layers, num_fc_neurons, d_ff)
model.load_state_dict(torch.load("./models/best_model_checkpoint.pth")['model_state_dict'])
model.eval()

batch_size = 1
sequence_length = 20
example_data = torch.randint(1, 50, (batch_size, sequence_length, 2)).float()
example_order = torch.randint(0, 10, (batch_size, sequence_length, 1)).sort(dim=1)[0].float()
print(example_order.shape)
example_input = torch.cat([example_data, example_order], dim=-1)

traced_model = torch.jit.trace(model, (example_input))
# traced_model = torch.jit.script(model)

now = datetime.now()
formatted_now = now.strftime("%Y-%m-%d_%H-%M-%S")

model_filename = f"bpp_{formatted_now}.pt"
traced_model.save(model_filename)